In [1]:
import pandas as pd
from functools import reduce
import operator
import ast

In [2]:
df = pd.read_csv("../data/tweets_data.csv")
df.head()

,account,retweet,hash_tags,user_mentions,tweet
0,rneelmani,False,"['IoT', 'business', 'IoT']","['Microsoft', 'dynamicCISO', 'dynamicCISO', 'D...","@Microsoft @dynamicCISO Among #IoT adopters, 8..."
1,rneelmani,False,"['IoT', 'IoT', 'IoT', 'InternetOfThings', 'Sen...","['Microsoft', 'dynamicCISO', 'dynamicCISO']",IoT Signals key findings:\n\n@Microsoft announ...
2,rneelmani,False,"['security', 'planes', 'hacking', 'aircraft', ...",['DHSgov'],The @DHSgov has issued a #security alert for s...
3,rneelmani,True,['ransomware'],"['dynamicCISO', 'rneelmani', 'reach2ratan']",A recent #ransomware attack impacting the data...
4,rneelmani,True,['Ransomware'],"['dynamicCISO', 'SonicWall']",@SonicWall #Ransomware Rising:\n\nDid you thin...


In [4]:
df["tweet"].values[10:20]

array(['@_Manish_Sharma_ bringing to your notice that @PanasonicIndia owes us a long overdue payment of INR 4.5 Lakh which is for participation in one of our events as a sponsor. Dozens of follow ups in person and on mail could not result into anything. Its unethical and unprofessional',
       '@s_v_g Agree. Ease of doing business is just a farce. In 8 years of entrepreneurship, I am yet to see an olive branch extended to me/business as a gesture. It’s always about how much more can we suck out of you!',
       "A man in #Canada recently tried to purchase #marijuana - using an ID card with @chrishemsworth's photo. Pictures of the obviously #fake ID, which put his name down as Thor Thunder Odinson, have gone viral online: https://t.co/RsO3I1ueIA #IdentityTheft #Breach @dynamicCISO",
       '@DynamicCIO announces the launch of its annual Enterprise #Innovation Summit 2019 (earlier knows as #CIO #Productivity Conclave). Also introducing Smart Innovator Felicitations this year. Follow the

In [3]:
df.shape

(2200, 5)

In [5]:
def clean(token):
    out = token.strip().replace('@', '').replace('#', '').replace('.', '').replace(',', '')\
                .replace('?', '').replace('!', '').replace(':', '').replace('\n', ' ')\
                .replace('%', ' percent ').replace('-', ' hyphen ')\
                .lower()
    out = out.split(' ')
    if '' in out:
        out.remove('')
    return out


def preprocess_tweet(tweet):
    tokens = tweet.split(' ')
    tokens1 = [clean(t) for t in tokens if 'https' not in t]
    try:
        tokens2 = reduce(operator.concat, tokens1)
    except:
        return tokens1
    return ' '.join(tokens2)


def preprocess_entities(entities):
    ents = ast.literal_eval(entities)
    out = list(set(ents))
    return '|'.join(out)

In [6]:
df["tweet"] = df["tweet"].apply(preprocess_tweet)
df["hash_tags"] = df["hash_tags"].apply(preprocess_entities)
df["user_mentions"] = df["user_mentions"].apply(preprocess_entities)
df.head()

,account,retweet,hash_tags,user_mentions,tweet
0,rneelmani,False,IoT|business,dynamicCISO|Microsoft|DynamicCIO,microsoft dynamicciso among iot adopters 88 pe...
1,rneelmani,False,InternetOfThings|IoT|Sensors,dynamicCISO|Microsoft,iot signals key findings microsoft announced i...
2,rneelmani,False,planes|criticalinfrastructure|hacking|aircraft...,DHSgov,the dhsgov has issued a security alert for sma...
3,rneelmani,True,ransomware,dynamicCISO|reach2ratan|rneelmani,a recent ransomware attack impacting the data ...
4,rneelmani,True,Ransomware,dynamicCISO|SonicWall,sonicwall ransomware rising did you think rans...


In [7]:
df.shape

(2200, 5)

In [8]:
df.to_csv("tweet_processed.csv", index=False)